In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from npj_utils import *
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

In [3]:
mimic_iv_path = "/cis/home/charr165/Documents/physionet.org/mimiciv/2.2"
mm_dir = "/cis/home/charr165/Documents/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [21]:
restrict_48_hours = False
include_notes = True
include_cxr = True
include_ecg = True
include_missing = True

In [22]:
import pickle

base_name = "pheno" # ihm, los, pheno
if restrict_48_hours:
    base_name += "-48"
else:
    base_name += "-all"

if include_cxr:
    if include_notes:
        base_name += "-cxr-notes"
    else:
        base_name += "-cxr"

if include_ecg:
    base_name += "-ecg"

if include_missing:
    base_name += "-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    train_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    val_stays = pickle.load(f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")

with open(f_path, "rb") as f:
    test_stays = pickle.load(f)

In [23]:
all_stays = train_stays + val_stays + test_stays

In [24]:
all_stays[0].keys()

dict_keys(['name', 'hadm_id', 'stay_id', 'ts_tt', 'irg_ts', 'irg_ts_mask', 'reg_ts', 'text_data', 'text_time_to_end', 'text_embeddings', 'text_missing', 'cxr_feats', 'cxr_time', 'cxr_missing', 'ecg_feats', 'ecg_time', 'ecg_missing', 'label'])

In [25]:
missing_keys = ['text_missing', 'cxr_missing', 'ecg_missing']

In [26]:
total_obs = {}
for key in missing_keys:
    total_obs[key] = 0

for stay in all_stays:
    for key in missing_keys:
        total_obs[key] += stay[key]

for key in missing_keys:
    total_obs[key] = len(all_stays) - total_obs[key]
total_obs

{'text_missing': 56824, 'cxr_missing': 14568, 'ecg_missing': 35925}

In [27]:
len(all_stays)

73173